In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022.csv')

# Check data types
print(data.dtypes)

# Handle missing values (if any)
# ... (Check for missing values and handle them appropriately)

# Encode categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[['gender']])
data = pd.concat([data, pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['gender']))], axis=1)


ID                    object
AGE                    int64
SEX                    int64
ALONE                float64
LOCOMOTION           float64
SENSORY              float64
VITALITY               int64
PSICHOLOGICAL        float64
COGNITION            float64
PHYSICAL_ACTIVITY    float64
SMOKE                float64
CIGARRETTES          float64
TOMA                 float64
DRINK                float64
CALC_IMC             float64
IMC                  float64
NURSING_HOME         float64
año                    int64
evaluacion             int64
dtype: object


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[columns_to_scale])
data[columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  for i in range(len(data)-look_back-1):
    person_data = data.iloc[i:(i+look_back), :]
    sequences.append(person_data.values)
    targets.append(data.iloc[i+look_back,data.columns.get_indexer(target_cols)])
  return np.array(sequences), np.array(targets)

# Create sequences and targets
look_back = 3
sequences, targets = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
pd.DataFrame(targets[0]).head()











,0
0,1.0
1,0.7
2,1.0
3,1.0
4,0.666667


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
import keras

# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC','AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psychologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psychological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psychologicals.append(psychological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psychologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[1]
sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape


# ARQUITECTURA

# Entrada secuencial (datos temporales)
sequential_input = layers.Input(shape=(sequences.shape[1], sequences.shape[2]))

# Entrada auxiliar (datos no temporales)
aux_input = layers.Input(shape=(auxiliares.shape[1],))


# Red LSTM para la entrada secuencial
x = layers.LSTM(24, return_sequences=True)(sequential_input)
x = layers.LSTM(24)(x)

# Concatenar la salida LSTM con las características auxiliares
x = layers.concatenate([x, aux_input])

# MLP para procesar la concatenación
shared_mlp = layers.Dense(8, activation='relu')(x)

# Rama para la Tarea 1 (Predicción de la puntuación Locomotion)
l=layers.Dense(8, activation='relu')(shared_mlp)  # capa intermedia
l_output = layers.Dense(1, activation='linear', name='l_output')(l)  # salida de regresión

# Rama para la Tarea 2 (Predicción de la puntuación Sensory)
s=layers.Dense(8, activation='relu')(shared_mlp)  # capa intermedia
s_output = layers.Dense(1, activation='linear', name='s_output')(s)  # salida de regresión

# Rama para la Tarea 3 (Predicción de la puntuación Vitality)
v=layers.Dense(8, activation='relu')(shared_mlp)  # capa intermedia
v_output = layers.Dense(1, activation='linear', name='v_output')(v)  # salida de regresión

# Rama para la Tarea 4 (Predicción de la puntuación Psichological)
p=layers.Dense(8, activation='relu')(shared_mlp)  # capa intermedia
p_output = layers.Dense(1, activation='linear', name='p_output')(p)  # salida de regresión

# Rama para la Tarea 5 (Predicción de la puntuación Cognition)
c=layers.Dense(8, activation='relu')(shared_mlp)  # capa intermedia
c_output = layers.Dense(1, activation='linear', name='c_output')(c)  # salida de regresión

# Salidas individuales para cada una de las 5 tareas
#outputs = []
#for i in range(5):
 #   y = layers.Dense(8, activation='relu')(mlp)  # capa intermedia
  #  y_output = layers.Dense(1, activation='linear', name=f'y{i+1}_output')(y)  # salida de regresión
   # outputs.append(y_output)

# Definir el modelo multitarea con entradas múltiples y salidas múltiples
# Modelo final con entradas y 5 salidas
model = models.Model(inputs=[sequential_input, aux_input], outputs=[l_output, s_output, v_output, p_output, c_output])

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse', 'mse', 'mse', 'mse', 'mse'],
              metrics=[['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae']])

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# 4. Entrenar el Modelo
history = model.fit([sequences, auxiliares],  # Entradas: secuencias y características auxiliares
                    {'l_output': locomotions, 's_output': sensories, 'v_output': vitalities, 'p_output': psichologicals, 'c_output': cognitions},  # Targets
                    epochs=100, batch_size=720, verbose=1)

# Evaluate the model
results = model.evaluate([sequences, auxiliares],
                         {'l_output': locomotions, 's_output': sensories, 'v_output': vitalities, 'p_output': psichologicals, 'c_output': cognitions})

# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
l_loss = results[1]  # Loss for 'l_output'
s_loss = results[2]  # Loss for 's_output'
v_loss = results[3]  # Loss for 'v_output'
p_loss = results[4]  # Loss for 'p_output'
c_loss = results[5]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
l_mae = results[6]  # MAE for 'l_output'
s_mae = results[7]  # MAE for 's_output'
v_mae = results[8]  # MAE for 'v_output'
p_mae = results[9]  # MAE for 'p_output'
c_mae = results[10]  # MAE for 'c_output'

# 5. Evaluar el Modelo
#loss, l_loss, s_loss, v_loss, p_loss, c_loss, l_mae, s_mae, v_mae, p_mae, c_mae = model.evaluate([sequences, auxiliares],
 #                     {'l_output': locomotions, 's_output': sensories, 'v_output': vitalities, 'p_output': psichologicals, 'c_output': cognitions})


#print(f"\nResultados de la evaluación:\n"
 #     f"Pérdida total: {loss:.4f}\n"
  #    f"Pérdida para l (Regresión): {l_loss:.4f}\n"
  #    f"Pérdida para s (Regresión): {s_loss:.4f}\n"
  #    f"Pérdida para v (Regresión): {v_loss:.4f}\n"
  #    f"Pérdida para p (Regresión): {p_loss:.4f}\n"
  #    f"Pérdida para c (Regresión): {c_loss:.4f}\n"
  #    f"MAE para l (Regresión): {l_mae:.4f}\n"
  #    f"MAE para s (Regresión): {s_mae:.4f}\n"
  #    f"MAE para v (Regresión): {v_mae:.4f}\n"
  #    f"MAE para p (Regresión): {p_mae:.4f}\n"
  #    f"MAE para c (Regresión): {c_mae:.4f}"
 #     )

#results = model.evaluate([sequences, auxiliares],
#                      {'l_output': locomotions, 's_output': sensories, 'v_output': vitalities, 'p_output': psichologicals, 'c_output': cognitions})

# Extract the loss and metrics
#loss = results[0]  # Total loss
#l_loss = results[1]  # Loss for 'l_output'
#s_loss = results[2]  # Loss for 's_output'
#v_loss = results[3]  # Loss for 'v_output'
#p_loss = results[4]  # Loss for 'p_output'
#c_loss = results[5]  # Loss for 'c_output'
#l_mae = results[6]   # MAE for 'l_output'
#s_mae = results[7]   # MAE for 's_output'
#v_mae = results[8]   # MAE for 'v_output'
#p_mae = results[9]   # MAE for 'p_output'
#c_mae = results[10]  # MAE for 'c_output'

print(f"\nResultados de la evaluación:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para l (Regresión): {l_loss:.4f}\n"
      f"Pérdida para s (Regresión): {s_loss:.4f}\n"
      f"Pérdida para v (Regresión): {v_loss:.4f}\n"
      f"Pérdida para p (Regresión): {p_loss:.4f}\n"
      f"Pérdida para c (Regresión): {c_loss:.4f}\n"
      f"MAE para l (Regresión): {l_mae:.4f}\n"
     f"MAE para s (Regresión): {s_mae:.4f}\n"
     f"MAE para v (Regresión): {v_mae:.4f}\n"
     f"MAE para p (Regresión): {p_mae:.4f}\n"
     f"MAE para c (Regresión): {c_mae:.4f}"
      )

model.save('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo2.keras')

!pip install netron
import netron
netron.start('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo2.keras')


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_8 (LSTM)             │ (None, 3, 24)          │          3,744 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_9 (LSTM)             │ (None, 24)             │          4,704 │ lstm_8[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 27)             │              0 │ lstm_9[0][0],          │
│ (Concatenate)             │                        │                │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 8)              │            224 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 8)              │             72 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_18 (Dense)          │ (None, 8)              │             72 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_19 (Dense)          │ (None, 8)              │             72 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_20 (Dense)          │ (None, 8)              │             72 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_21 (Dense)          │ (None, 8)              │             72 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l_output (Dense)          │ (None, 1)              │              9 │ dense_17[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ s_output (Dense)          │ (None, 1)              │              9 │ dense_18[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ v_output (Dense)          │ (None, 1)              │              9 │ dense_19[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_output (Dense)          │ (None, 1)              │              9 │ dense_20[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ c_output (Dense)          │ (None, 1)              │              9 │ dense_21[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,077 (35.46 KB)

 Trainable params: 9,077 (35.46 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - c_output_mae: 0.7480 - c_output_mse: 0.5784 - l_output_mae: 0.8336 - l_output_mse: 0.7523 - loss: 3.4149 - p_output_mae: 0.9080 - p_output_mse: 0.8590 - s_output_mae: 0.6293 - s_output_mse: 0.4263 - v_output_mae: 0.8795 - v_output_mse: 0.7989
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - c_output_mae: 0.7522 - c_output_mse: 0.5845 - l_output_mae: 0.8023 - l_output_mse: 0.7015 - loss: 3.2502 - p_output_mae: 0.8862 - p_output_mse: 0.8187 - s_output_mae: 0.6133 - s_output_mse: 0.4075 - v_output_mae: 0.8448 - v_output_mse: 0.7380
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - c_output_mae: 0.7647 - c_output_mse: 0.6041 - l_output_mae: 0.7520 - l_output_mse: 0.6199 - loss: 3.1146 - p_output_mae: 0.8676 - p_output_mse: 0.7846 - s_output_mae: 0.5865 - s_output_mse: 0.3735 - v_output_mae: 0.8408 - v_output_mse: 0.7324
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - c_output_mae: 0.7799 - c_output_mse: 0.6274 - l_output_mae: 0.

('localhost', 17892)

STRATEGY 2. CODE ALTERNATIVE

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam optimizer
%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psychologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psychological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psychologicals.append(psychological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psychologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    l_output = Dense(1, activation='linear', name='l_output')(shared_mlp)
    s_output = Dense(1, activation='linear', name='s_output')(shared_mlp)
    v_output = Dense(1, activation='linear', name='v_output')(shared_mlp)
    p_output = Dense(1, activation='linear', name='p_output')(shared_mlp)
    c_output = Dense(1, activation='linear', name='c_output')(shared_mlp)

    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[l_output, s_output, v_output, p_output, c_output ])
    return model

# Compile the model
model = create_model()
#optimizer = Adam(learning_rate=0.001)
model.compile(optimizer='sgd',
              loss=['mse', 'mse', 'mse', 'mse', 'mse'],
              metrics=[['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae']])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences, auxiliares], y=[locomotions, sensories, vitalities, psichologicals, cognitions], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences, auxiliares],
                        {'l_output': locomotions, 's_output': sensories, 'v_output': vitalities, 'p_output': psichologicals, 'c_output': cognitions})

# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
l_loss = results[1]  # Loss for 'l_output'
s_loss = results[2]  # Loss for 's_output'
v_loss = results[3]  # Loss for 'v_output'
p_loss = results[4]  # Loss for 'p_output'
c_loss = results[5]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
l_mae = results[6]  # MAE for 'l_output'
s_mae = results[7]  # MAE for 's_output'
v_mae = results[8]  # MAE for 'v_output'
p_mae = results[9]  # MAE for 'p_output'
c_mae = results[10]  # MAE for 'c_output'


print(f"\nResultados de la evaluación:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para l (Regresión): {l_loss:.4f}\n"
      f"Pérdida para s (Regresión): {s_loss:.4f}\n"
      f"Pérdida para v (Regresión): {v_loss:.4f}\n"
      f"Pérdida para p (Regresión): {p_loss:.4f}\n"
      f"Pérdida para c (Regresión): {c_loss:.4f}\n"
      f"MAE para l (Regresión): {l_mae:.4f}\n"
     f"MAE para s (Regresión): {s_mae:.4f}\n"
    f"MAE para v (Regresión): {v_mae:.4f}\n"
     f"MAE para p (Regresión): {p_mae:.4f}\n"
     f"MAE para c (Regresión): {c_mae:.4f}"
      )

#%tensorboard --logdir logs

model.save('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')

!pip install netron
import netron
netron.start('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-4-ce905c556069>:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-4-ce905c556069>:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 24)             │              0 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 27)             │              0 │ dropout_2[0][0],       │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 8)              │            224 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 8)              │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l_output (Dense)          │ (None, 1)              │              9 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ s_output (Dense)          │ (None, 1)              │              9 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ v_output (Dense)          │ (None, 1)              │              9 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_output (Dense)          │ (None, 1)              │              9 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ c_output (Dense)          │ (None, 1)              │              9 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,013 (15.68 KB)

 Trainable params: 4,013 (15.68 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - c_output_loss: 0.3987 - c_output_mae: 0.5639 - c_output_mse: 0.4001 - l_output_loss: 0.6538 - l_output_mae: 0.7569 - l_output_mse: 0.6567 - loss: 2.4812 - p_output_loss: 0.5337 - p_output_mae: 0.6289 - p_output_mse: 0.5360 - s_output_loss: 0.3236 - s_output_mae: 0.4716 - s_output_mse: 0.3253 - v_output_loss: 0.5609 - v_output_mae: 0.6722 - v_output_mse: 0.5631
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - c_output_loss: 0.0741 - c_output_mae: 0.2141 - c_output_mse: 0.0742 - l_output_loss: 0.1722 - l_output_mae: 0.3728 - l_output_mse: 0.1728 - loss: 0.6708 - p_output_loss: 0.2038 - p_output_mae: 0.3581 - p_output_mse: 0.2043 - s_output_loss: 0.1051 - s_output_mae: 0.2459 - s_output_mse: 0.1051 - v_output_loss: 0.1142 - v_output_mae: 0.2697 - v_output_mse: 0.1143
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - c_output_loss: 0.0666 - c_output_mae: 0.2017 - c_output_mse: 0.0666 - l_output_loss: 0.1040 - l_output_mae: 0.2738 

('localhost', 8081)

In [ ]:
import netron
netron.start('mi_modelo.keras')

Serving 'mi_modelo.keras' at http://localhost:23121


('localhost', 23121)

DATA DE PRUEBA 80% PARA ENTRENAR 20% PARA PROBAR

In [ ]:
!pip install scikit-learn==0.23.2

  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
!pip install scikit-learn==0.24.0 # Install a compatible version of scikit-learn
!pip install eli5 --upgrade # Upgrade eli5 to ensure compatibility

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
Failed to build scikit-learn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (scikit-learn)


In [ ]:
!pip install eli5 --upgrade

In [ ]:
!pip install shap

MULTITASK LSTM MODEL 80% TRAIN 20% TEST NO XAI

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape



# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    l_output = Dense(1, activation='linear', name='l_output')(shared_mlp)
    s_output = Dense(1, activation='linear', name='s_output')(shared_mlp)
    v_output = Dense(1, activation='linear', name='v_output')(shared_mlp)
    p_output = Dense(1, activation='linear', name='p_output')(shared_mlp)
    c_output = Dense(1, activation='linear', name='c_output')(shared_mlp)

    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[l_output, s_output, v_output, p_output, c_output ])
    return model

# Compile the model
model = create_model()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse', 'mse', 'mse', 'mse', 'mse'],
              metrics=[['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae']])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[locomotions_train, sensories_train, vitalities_train, psichologicals_train, cognitions_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        {'l_output': locomotions_test, 's_output': sensories_test, 'v_output': vitalities_test, 'p_output': psichologicals_test, 'c_output': cognitions_test})

# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
l_loss = results[1]  # Loss for 'l_output'
s_loss = results[2]  # Loss for 's_output'
v_loss = results[3]  # Loss for 'v_output'
p_loss = results[4]  # Loss for 'p_output'
c_loss = results[5]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
l_mae = results[6]  # MAE for 'l_output'
s_mae = results[7]  # MAE for 's_output'
v_mae = results[8]  # MAE for 'v_output'
p_mae = results[9]  # MAE for 'p_output'
c_mae = results[10]  # MAE for 'c_output'


print(f"\nResultados de la evaluación:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para l (Regresión): {l_loss:.4f}\n"
      f"Pérdida para s (Regresión): {s_loss:.4f}\n"
      f"Pérdida para v (Regresión): {v_loss:.4f}\n"
      f"Pérdida para p (Regresión): {p_loss:.4f}\n"
      f"Pérdida para c (Regresión): {c_loss:.4f}\n"
      f"MAE para l (Regresión): {l_mae:.4f}\n"
     f"MAE para s (Regresión): {s_mae:.4f}\n"
    f"MAE para v (Regresión): {v_mae:.4f}\n"
     f"MAE para p (Regresión): {p_mae:.4f}\n"
     f"MAE para c (Regresión): {c_mae:.4f}"
      )

#%tensorboard --logdir logs

model.save('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')

!pip install netron
import netron
netron.start('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-14-70a9fd16e4b0>:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-14-70a9fd16e4b0>:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_12 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_24 (Dropout)      │ (None, 24)             │              0 │ lstm_12[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_12            │ (None, 27)             │              0 │ dropout_24[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 8)              │            224 │ concatenate_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 8)              │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l_output (Dense)          │ (None, 1)              │              9 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ s_output (Dense)          │ (None, 1)              │              9 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ v_output (Dense)          │ (None, 1)              │              9 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_output (Dense)          │ (None, 1)              │              9 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ c_output (Dense)          │ (None, 1)              │              9 │ dropout_25[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,013 (15.68 KB)

 Trainable params: 4,013 (15.68 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


LookupError: gradient registry has no entry for: shap_AddV2

MULTITASK LSTM (FULL MODEL) -> 80% TRAIN / 20% TEST -> EXPLAINABILITY

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape



# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    l_output = Dense(1, activation='linear', name='l_output')(shared_mlp)
    s_output = Dense(1, activation='linear', name='s_output')(shared_mlp)
    v_output = Dense(1, activation='linear', name='v_output')(shared_mlp)
    p_output = Dense(1, activation='linear', name='p_output')(shared_mlp)
    c_output = Dense(1, activation='linear', name='c_output')(shared_mlp)

    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[l_output, s_output, v_output, p_output, c_output ])
    return model

# Compile the model
model = create_model()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse', 'mse', 'mse', 'mse', 'mse'],
              metrics=[['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae'], ['mse','mae']])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[locomotions_train, sensories_train, vitalities_train, psichologicals_train, cognitions_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        {'l_output': locomotions_test, 's_output': sensories_test, 'v_output': vitalities_test, 'p_output': psichologicals_test, 'c_output': cognitions_test})

# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
l_loss = results[1]  # Loss for 'l_output'
s_loss = results[2]  # Loss for 's_output'
v_loss = results[3]  # Loss for 'v_output'
p_loss = results[4]  # Loss for 'p_output'
c_loss = results[5]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
l_mae = results[6]  # MAE for 'l_output'
s_mae = results[7]  # MAE for 's_output'
v_mae = results[8]  # MAE for 'v_output'
p_mae = results[9]  # MAE for 'p_output'
c_mae = results[10]  # MAE for 'c_output'


print(f"\nResultados de la evaluación:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para l (Regresión): {l_loss:.4f}\n"
      f"Pérdida para s (Regresión): {s_loss:.4f}\n"
      f"Pérdida para v (Regresión): {v_loss:.4f}\n"
      f"Pérdida para p (Regresión): {p_loss:.4f}\n"
      f"Pérdida para c (Regresión): {c_loss:.4f}\n"
      f"MAE para l (Regresión): {l_mae:.4f}\n"
     f"MAE para s (Regresión): {s_mae:.4f}\n"
    f"MAE para v (Regresión): {v_mae:.4f}\n"
     f"MAE para p (Regresión): {p_mae:.4f}\n"
     f"MAE para c (Regresión): {c_mae:.4f}"
      )

#%tensorboard --logdir logs

model.save('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')

!pip install netron
import netron
netron.start('/content/drive/My Drive/PREDICTIVE MODEL IC/mi_modelo.keras')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-12-35e6b1e9de46>:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-12-35e6b1e9de46>:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_10 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 24)             │              0 │ lstm_10[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_10            │ (None, 27)             │              0 │ dropout_20[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 8)              │            224 │ concatenate_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_21 (Dropout)      │ (None, 8)              │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l_output (Dense)          │ (None, 1)              │              9 │ dropout_21[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ s_output (Dense)          │ (None, 1)              │              9 │ dropout_21[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ v_output (Dense)          │ (None, 1)              │              9 │ dropout_21[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_output (Dense)          │ (None, 1)              │              9 │ dropout_21[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ c_output (Dense)          │ (None, 1)              │              9 │ dropout_21[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,013 (15.68 KB)

 Trainable params: 4,013 (15.68 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


LookupError: gradient registry has no entry for: shap_AddV2